# Ubicaciones.xlsx -> (ubi_cliente)

In [1]:
import re
import numpy as np
import logging
import pandas as pd

import warnings
warnings.filterwarnings('ignore')
import unicodedata

In [2]:
# 1. Leer el archivo Excel
print("1. Leyendo el archivo Excel...")
ruta = "../Sucia/ubicaciones.xlsx"
df_ubicaciones = pd.read_excel(ruta, sheet_name='Ubicacion', engine='openpyxl')
print("Datos leídos con éxito. Muestra de datos antes de la limpieza:")
df_ubicaciones.info()
df_ubicaciones.head(2)

1. Leyendo el archivo Excel...
Datos leídos con éxito. Muestra de datos antes de la limpieza:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3127 entries, 0 to 3126
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   CODIGO     748 non-null    object        
 1   CODIGO2    747 non-null    object        
 2   UBICACIÓN  747 non-null    object        
 3   FECHA      748 non-null    datetime64[ns]
 4   ORIGEN     748 non-null    object        
 5   CODCLI2    3127 non-null   object        
 6   TAREAS     2915 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 171.1+ KB


,CODIGO,CODIGO2,UBICACIÓN,FECHA,ORIGEN,CODCLI2,TAREAS
0,80313,"80313,SOL Y AGU","http://maps.google.com/?q=-34.770816,-55.575468",2024-12-14,agregados_adrian,80313,80313.0
1,80315,"80315,CASA NEPTUNO","http://maps.google.com/?q=-34.782554,-55.556325",2024-12-14,agregados_adrian,80315,80315.0


## BLOQUE 1: CARGA INICIAL Y LIMPIEZA BÁSICA

In [3]:
print("\n=== BLOQUE 1: CARGA Y LIMPIEZA BÁSICA ===")
print("--------------------------------------------")

# 1.1. Carga inicial del archivo
try:
    ruta = "../Sucia/ubicaciones.xlsx"
    df_ubicaciones = pd.read_excel(ruta, sheet_name='Ubicacion', engine='openpyxl')
    print("✓ Archivo de ubicaciones cargado exitosamente")
    total_filas_inicial = df_ubicaciones.shape[0]
    print(f"Número total de filas al inicio: {total_filas_inicial}")
except Exception as e:
    print(f"❌ Error al cargar archivo: {e}")
    raise

# 1.2. Eliminar filas completamente vacías
print("\n1.2. Eliminando filas completamente vacías...")
filas_antes = df_ubicaciones.shape[0]
df_ubicaciones = df_ubicaciones.dropna(how='all').copy()
print(f"Filas eliminadas: {filas_antes - df_ubicaciones.shape[0]}")
print(f"Filas restantes: {df_ubicaciones.shape[0]}")

# 1.3. Eliminar filas mayoritariamente vacías (80% o más)
print("\n1.3. Eliminando filas mayoritariamente vacías...")
columnas_verificar = df_ubicaciones.columns.tolist()
umbral_vacias = 0.8  # 80% de columnas vacías o con valor 0

condicion_mayoritariamente_vacia = (
    (df_ubicaciones[columnas_verificar].isnull().sum(axis=1) > len(columnas_verificar) * umbral_vacias) |
    ((df_ubicaciones[columnas_verificar].isnull() | df_ubicaciones[columnas_verificar].eq(0.0)).sum(axis=1) > len(columnas_verificar) * umbral_vacias)
)

total_filas_mayoritariamente_vacias = condicion_mayoritariamente_vacia.sum()
print(f"Filas mayoritariamente vacías a eliminar (>80% vacías o 0.0): {total_filas_mayoritariamente_vacias}")

df_ubicaciones = df_ubicaciones[~condicion_mayoritariamente_vacia].copy()
print(f"Filas restantes: {df_ubicaciones.shape[0]}")

# 1.4. Verificación de columna TAREAS
print("\n1.4. Verificando columna 'TAREAS'...")
if 'TAREAS' in df_ubicaciones.columns:
    filas_vacias_tareas = df_ubicaciones['TAREAS'].isna() | (df_ubicaciones['TAREAS'].astype(str).str.strip() == "")
    total_filas_vacias_tareas = filas_vacias_tareas.sum()
    print(f"Filas con 'TAREAS' vacías: {total_filas_vacias_tareas}")
    df_ubicaciones = df_ubicaciones[~filas_vacias_tareas].copy()
    print(f"Filas restantes: {df_ubicaciones.shape[0]}")
else:
    print("La columna 'TAREAS' no existe en el DataFrame")


=== BLOQUE 1: CARGA Y LIMPIEZA BÁSICA ===
--------------------------------------------
✓ Archivo de ubicaciones cargado exitosamente
Número total de filas al inicio: 3127

1.2. Eliminando filas completamente vacías...
Filas eliminadas: 0
Filas restantes: 3127

1.3. Eliminando filas mayoritariamente vacías...
Filas mayoritariamente vacías a eliminar (>80% vacías o 0.0): 2379
Filas restantes: 748

1.4. Verificando columna 'TAREAS'...
Filas con 'TAREAS' vacías: 212
Filas restantes: 536


## BLOQUE 2:  PROCESAMIENTO DE NOMBRES Y CÓDIGOS 

**Observaciones df_ubicaciones** 
- La columna CODIGO2, incluye nombres, vamos a separarlos en una nueva columna con el nombre NOMCLI.
- La columna UBICACION contiene los enlaces con coordenadas a Google Map podriamos proceder creando una columna Latitud y otra Longitud.
  
**Procedemos a ejecutar lo descrito:**

In [4]:
print("\n=== BLOQUE 2: PROCESAMIENTO DE NOMBRES Y CÓDIGOS ===")
print("------------------------------------------------")

def clean_name(name):
    if pd.isna(name):
        return ""
    name = re.sub(r'[^\w ,]+', ' ', name, flags=re.UNICODE)
    name = re.sub(r'\bNOMBRE\b', '', name, flags=re.IGNORECASE)
    name = re.sub(r'\s+', ' ', name).strip()
    return name

filas_sin_nombre = []

def extract_name_and_update_code(row):
    code_str = str(row['CODIGO2']).strip() if not pd.isna(row['CODIGO2']) else ''
    codcli_str = str(row['CODIGO']).strip() if not pd.isna(row['CODIGO']) else ''
    
    extracted_code = None
    name_part = None
    combined_str = f"{code_str} {codcli_str}".strip()
    
    if code_str == codcli_str and '.' in code_str:
        parts = code_str.split('.', 1)
        if len(parts) == 2:
            extracted_code = parts[0].strip()
            name_part = parts[1].strip()
            name_part = clean_name(name_part)
            if not name_part:
                name_part = "SIN NOMBRE"
                filas_sin_nombre.append(row.name)
            return pd.Series({'NOMCLI': name_part, 'CODIGO2': extracted_code, 'CODIGO': extracted_code})

    codes = re.findall(r'\b(\d{5,})\b', combined_str)
    if codes:
        extracted_code = codes[0]
        pattern = r'[\W_]*\b{}\b[\W_]*'.format(re.escape(extracted_code))
        name_part = re.sub(pattern, ' ', combined_str)
        name_part = name_part.strip(" .;,*-_").strip()
        if name_part:
            name_part = clean_name(name_part)
            if not name_part:
                name_part = "SIN NOMBRE"
                filas_sin_nombre.append(row.name)
        else:
            name_part = "SIN NOMBRE"
            filas_sin_nombre.append(row.name)
    else:
        extracted_code = None
        name_part = code_str or codcli_str or "SIN NOMBRE"
        name_part = name_part.strip(" .;,*-_").strip()
        name_part = clean_name(name_part)
        if not name_part:
            name_part = "SIN NOMBRE"
            filas_sin_nombre.append(row.name)

    return pd.Series({'NOMCLI': name_part, 'CODIGO2': extracted_code, 'CODIGO': extracted_code})


print("2.1. Aplicando procesamiento de nombres y códigos...")
df_ubicaciones[['NOMCLI', 'CODIGO2', 'CODIGO']] = df_ubicaciones.apply(extract_name_and_update_code, axis=1)

print("\n2.2. Verificando tipos y valores de CODIGO...")
print(f"Tipo de datos de CODIGO: {df_ubicaciones['CODIGO'].dtype}")

non_numeric_codes = df_ubicaciones[~df_ubicaciones['CODIGO'].astype(str).str.fullmatch(r'\d+')]
total_non_numeric = non_numeric_codes.shape[0]
print(f"Valores no numéricos en CODIGO: {total_non_numeric}")


=== BLOQUE 2: PROCESAMIENTO DE NOMBRES Y CÓDIGOS ===
------------------------------------------------
2.1. Aplicando procesamiento de nombres y códigos...

2.2. Verificando tipos y valores de CODIGO...
Tipo de datos de CODIGO: object
Valores no numéricos en CODIGO: 0


## BLOQUE 3: PROCESAMIENTO DE UBICACIONES

In [5]:
def extract_coordinates(ubicacion):
    if pd.isna(ubicacion):
        return pd.Series({
            'LATITUD': None, 
            'LONGITUD': None, 
            'RAZON_COORDENADAS': 'Ubicación nula'
        })

    ubicacion = str(ubicacion)
    patterns = [
        r'lat/lng:\s*\(\s*(-?\d+\.\d+),\s*(-?\d+\.\d+)\s*\)',
        r'lat/lng:\s*(-?\d+\.\d+),\s*(-?\d+\.\d+)',
        r'http[s]?://maps\.google\.com/\?q\s*=\s*(-?\d+\.\d+)\s*,\s*(-?\d+\.\d+)',
        r'@(-?\d+\.\d+),\s*(-?\d+\.\d+)',
        r'q=(-?\d+\.\d+),\s*(-?\d+\.\d+)',
        r'!3d\s*(-?\d+\.\d+)!4d\s*(-?\d+\.\d+)',
        r'https?://maps\.app\.goo\.gl/[^?]*\?q=(-?\d+\.\d+),\s*(-?\d+\.\d+)',
    ]

    for pattern in patterns:
        match = re.search(pattern, ubicacion)
        if match:
            try:
                lat = float(match.group(1))
                lon = float(match.group(2))
                return pd.Series({
                    'LATITUD': lat, 
                    'LONGITUD': lon, 
                    'RAZON_COORDENADAS': 'Coordenadas extraídas'
                })
            except ValueError:
                return pd.Series({
                    'LATITUD': None, 
                    'LONGITUD': None, 
                    'RAZON_COORDENADAS': 'Error en conversión de coordenadas'
                })

    # Búsqueda de coordenadas adicional
    all_coords = re.findall(r'(-?\d+\.\d+)', ubicacion)
    if len(all_coords) >= 2:
        try:
            lat = float(all_coords[0])
            lon = float(all_coords[1])
            return pd.Series({
                'LATITUD': lat, 
                'LONGITUD': lon, 
                'RAZON_COORDENADAS': 'Coordenadas alternativas extraídas'
            })
        except ValueError:
            pass

    return pd.Series({
        'LATITUD': None, 
        'LONGITUD': None, 
        'RAZON_COORDENADAS': 'No se encontraron coordenadas'
    })

# Proceso de extracción
df_ubicaciones[['LATITUD', 'LONGITUD', 'RAZON_COORDENADAS']] = df_ubicaciones['UBICACIÓN'].apply(extract_coordinates)

# Análisis de resultados
print("Resumen de extracción de coordenadas:")
print(df_ubicaciones['RAZON_COORDENADAS'].value_counts())

# Exportar filas problemáticas
filas_problematicas = df_ubicaciones[df_ubicaciones['LATITUD'].isna()]
filas_problematicas.to_excel('ubicaciones_sin_coordenadas.xlsx', index=False)

Resumen de extracción de coordenadas:
RAZON_COORDENADAS
Coordenadas extraídas            527
No se encontraron coordenadas      8
Ubicación nula                     1
Name: count, dtype: int64


In [6]:
filas_problematicas.head(8)

,CODIGO,CODIGO2,UBICACIÓN,FECHA,ORIGEN,CODCLI2,TAREAS,NOMCLI,LATITUD,LONGITUD,RAZON_COORDENADAS
118,12920,12920,https://maps.app.goo.gl/YFZK9vEkaZbNyrxy5?g_st=ac,2024-10-16,agregados_adrian,12920,12920.0,Entregar Pilecord,NaN,NaN,No se encontraron coordenadas
152,100006,100006,https://maps.app.goo.gl/RdKDZfi5WDGPKMkt9,2024-09-05,agregados_adrian,100006,100006.0,CASA LAUTARO,NaN,NaN,No se encontraron coordenadas
185,100000,100000,https://maps.app.goo.gl/TJdD7rTMyTRzzCbY7,2024-08-07,agregados_adrian,100000,100000.0,EMPRESA,NaN,NaN,No se encontraron coordenadas
186,100001,100001,https://maps.app.goo.gl/RYAzoCFef6kyb2sb7,2024-08-07,agregados_adrian,100001,100001.0,CASA DARIO,NaN,NaN,No se encontraron coordenadas
187,100002,100002,NaN,2024-08-07,agregados_adrian,100002,100002.0,CASA HUGO,NaN,NaN,Ubicación nula
188,100003,100003,https://maps.app.goo.gl/cJZv8DexaNKHfXJN6,2024-08-07,agregados_adrian,100003,100003.0,CASA CONRADO,NaN,NaN,No se encontraron coordenadas
189,100004,100004,https://maps.app.goo.gl/VAukkvZeqQn3iVS48,2024-08-07,agregados_adrian,100004,100004.0,CASA MIGUEL,NaN,NaN,No se encontraron coordenadas
190,100005,100005,https://maps.app.goo.gl/uBJ6PMaBsF6tTWXz9,2024-08-07,agregados_adrian,100005,100005.0,CASA MARTIN,NaN,NaN,No se encontraron coordenadas


### Agregamos:
- 12920 
- 82177

Los demas codigos(100001 a 100006) son referidos a ubicacion empresa y los tenemos en nuestro set **ubi_empresa.xlsx**

In [7]:

#Imprimimos las filas con los codigos 12920 y 82177
print(df_ubicaciones.loc[df_ubicaciones['CODIGO'].isin(['12920', '82177'])])


# Diccionario de ubicaciones manuales
ubicaciones_manual = {
  12920: "https://www.google.com/maps/place/34%C2%B046'42.6%22S+55%C2%B051'36.3%22W/@-34.778492,-55.860085,17z/data=!3m1!4b1!4m4!3m3!8m2!3d-34.778492!4d-55.860085?entry=ttu&g_ep=4EgoyMDI0MTAyOS4wIKXMDSoASAFQAw%3D%D",
  82177: "https://www.google.com/google.com/maps/place/-34.778100,-55.862742/data=!4m6!3m5!1s0!7e2!8m2!3d-34.7780997!4d-55.8627423?utm_source=mstt_1&entry=gps"
}

# Asignar las ubicaciones
for codigo, ubicacion in ubicaciones_manual.items():
  df_ubicaciones.loc[df_ubicaciones['CODIGO'] == codigo, 'UBICACIÓN'] = ubicacion


    CODIGO CODIGO2                                          UBICACIÓN  \
118  12920   12920  https://maps.app.goo.gl/YFZK9vEkaZbNyrxy5?g_st=ac   
247  82177   82177  Marcador\n[Google Maps](https://maps.app.goo.g...   

         FECHA            ORIGEN CODCLI2   TAREAS  \
118 2024-10-16  agregados_adrian   12920  12920.0   
247 2024-04-27                 1   82177  82177.0   

                                  NOMCLI  LATITUD  LONGITUD  \
118                    Entregar Pilecord      NaN       NaN   
247  SEBASTIAN PALACIOS,24 HORAS PINAMAR      NaN       NaN   

                 RAZON_COORDENADAS  
118  No se encontraron coordenadas  
247  No se encontraron coordenadas  


### Una vez arreglados corroboramos:

In [8]:
#Imprimimos las filas con los codigos 12920 y 82177
df_ubicaciones.loc[df_ubicaciones['CODIGO'].isin(['12920', '82177'])]

,CODIGO,CODIGO2,UBICACIÓN,FECHA,ORIGEN,CODCLI2,TAREAS,NOMCLI,LATITUD,LONGITUD,RAZON_COORDENADAS
118,12920,12920,https://maps.app.goo.gl/YFZK9vEkaZbNyrxy5?g_st=ac,2024-10-16,agregados_adrian,12920,12920.0,Entregar Pilecord,NaN,NaN,No se encontraron coordenadas
247,82177,82177,Marcador\n[Google Maps](https://maps.app.goo.g...,2024-04-27,1,82177,82177.0,"SEBASTIAN PALACIOS,24 HORAS PINAMAR",NaN,NaN,No se encontraron coordenadas


## BLOQUE 4: LIMPIEZA FINAL Y GUARDAD

In [9]:

print("\n=== BLOQUE 4: LIMPIEZA FINAL Y GUARDADO ===")
print("-----------------------------------------")

# 4.1. Crear DataFrame final
print("\n4.1. Creando DataFrame final...")
ubi_cliente = df_ubicaciones[['CODIGO', 'NOMCLI', 'LATITUD', 'UBICACIÓN', 'LONGITUD', 'FECHA']].copy()

# 4.2. Conversiones finales
print("\n4.2. Realizando conversiones finales...")
ubi_cliente['LONGITUD'] = pd.to_numeric(ubi_cliente['LONGITUD'], errors='coerce')
ubi_cliente['LATITUD'] = pd.to_numeric(ubi_cliente['LATITUD'], errors='coerce')
ubi_cliente['NOMCLI'] = ubi_cliente['NOMCLI'].astype(str)

# 4.3. Limpiezas finales
print("\n4.3. Realizando limpiezas finales...")
filas_antes_limpieza = ubi_cliente.shape[0]
ubi_cliente = ubi_cliente.dropna(subset=['LATITUD', 'LONGITUD', 'UBICACIÓN', 'CODIGO', 'NOMCLI'])
ubi_cliente = ubi_cliente.drop_duplicates(subset=['NOMCLI'])
ubi_cliente.reset_index(drop=True, inplace=True)

# 4.4. Reportes finales
print("\n4.4. Generando reportes finales...")
zero_coords = ubi_cliente[(ubi_cliente['LATITUD'] == 0) & (ubi_cliente['LONGITUD'] == 0)]
print(f"Registros con coordenadas en 0: {len(zero_coords)}")
print(f"Registros sin nombre: {len(filas_sin_nombre)}")

# 4.5. Guardar resultados
print("\n4.5. Guardando resultados...")
try:
    ruta_salida = "../Limpia/Ubicaciones_direcciones.xlsx"
    with pd.ExcelWriter(ruta_salida) as writer:
        ubi_cliente.to_excel(writer, sheet_name='Ubicaciones_Limpias', index=False)
        if not zero_coords.empty:
            zero_coords.to_excel(writer, sheet_name='Coordenadas_En_0', index=False)
    print(f"✓ Archivo guardado exitosamente en: {ruta_salida}")
except Exception as e:
    print(f"❌ Error al guardar: {e}")
    raise

print("\n=== PROCESO COMPLETADO ===")
print(f"Registros iniciales: {total_filas_inicial}")
print(f"Registros finales: {len(ubi_cliente)}")
print(f"Registros eliminados: {total_filas_inicial - len(ubi_cliente)}")



=== BLOQUE 4: LIMPIEZA FINAL Y GUARDADO ===
-----------------------------------------

4.1. Creando DataFrame final...

4.2. Realizando conversiones finales...

4.3. Realizando limpiezas finales...

4.4. Generando reportes finales...
Registros con coordenadas en 0: 0
Registros sin nombre: 0

4.5. Guardando resultados...
✓ Archivo guardado exitosamente en: ../Limpia/Ubicaciones_direcciones_original.xlsx

=== PROCESO COMPLETADO ===
Registros iniciales: 3127
Registros finales: 512
Registros eliminados: 2615


In [10]:
df_ubicaciones.info()
df_ubicaciones.head(5)

<class 'pandas.core.frame.DataFrame'>
Index: 536 entries, 0 to 747
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   CODIGO             536 non-null    object        
 1   CODIGO2            536 non-null    object        
 2   UBICACIÓN          535 non-null    object        
 3   FECHA              536 non-null    datetime64[ns]
 4   ORIGEN             536 non-null    object        
 5   CODCLI2            536 non-null    object        
 6   TAREAS             536 non-null    float64       
 7   NOMCLI             536 non-null    object        
 8   LATITUD            527 non-null    float64       
 9   LONGITUD           527 non-null    float64       
 10  RAZON_COORDENADAS  536 non-null    object        
dtypes: datetime64[ns](1), float64(3), object(7)
memory usage: 50.2+ KB


,CODIGO,CODIGO2,UBICACIÓN,FECHA,ORIGEN,CODCLI2,TAREAS,NOMCLI,LATITUD,LONGITUD,RAZON_COORDENADAS
0,80313,80313,"http://maps.google.com/?q=-34.770816,-55.575468",2024-12-14,agregados_adrian,80313,80313.0,SOL Y AGU,-34.770816,-55.575468,Coordenadas extraídas
1,80315,80315,"http://maps.google.com/?q=-34.782554,-55.556325",2024-12-14,agregados_adrian,80315,80315.0,CASA NEPTUNO,-34.782554,-55.556325,Coordenadas extraídas
2,80309,80309,"http://maps.google.com/?q=-34.734095,-55.767212",2024-12-14,agregados_adrian,80309,80309.0,LOS HERMANOS,-34.734095,-55.767212,Coordenadas extraídas
3,80314,80314,"http://maps.google.com/?q=-34.761454,-55.747426",2024-12-14,agregados_adrian,80314,80314.0,MARALI,-34.761454,-55.747426,Coordenadas extraídas
4,81104,81104,"http://maps.google.com/?q=-34.764847,-55.825276",2024-12-14,agregados_adrian,81104,81104.0,EL COMIENZO,-34.764847,-55.825276,Coordenadas extraídas
